In [1]:
import os

os.chdir("/root/dev/human/")
os.getcwd()

'/root/dev/human'

In [2]:
import torch

torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!gpustat

b4d440a4085d                Mon Oct  7 21:58:18 2024  550.100
[0] NVIDIA GeForce RTX 4090 | 38°C,   0 % |   396 / 24564 MB |


# Selective scan

In [ ]:
def selective_state_update(state, x, dt, A, B, C, D=None, z=None, dt_bias=None, dt_softplus=False):
    has_heads = state.dim() > 3
    if state.dim() == 3:
        state = state.unsqueeze(1) # (B, num_h, D, H) → H might be dimension of state (dstate, originally)
    if x.dim() == 2:
        x = x.unsqueeze(1) # (B, num_h, D) → number of heads
    if dt.dim() == 2:
        dt = dt.unsqueeze(1) # (B, num_h, D)
    if A.dim() == 2:
        A = A.unsqueeze(0) # (D, H)
    if B.dim() == 2:
        B = B.unsqueeze(1) # (B, num_groups, H)
    if C.dim() == 2:
        C = C.unsqueeze(1) # (B, num_groups, H)
    if D is not None and D.dim() == 1:
        D = D.unsqueeze(0) # (num_h, D)
    if z is not None and z.dim() == 2:
        z = z.unsqueeze(1)
    if dt_bias is not None and dt_bias.dim() == 1:
        dt_bias = dt_bias.unsqueeze(0)
    
    batch, nheads, dim, dstate = state.shape
    assert x.shape == (batch, nheads, dim)
    assert dt.shape == x.shape
    assert A.shape == (nheads, dim, dstate)
    ngroups = B.shape[1]
    assert nheads % ngroups == 0, "nheads must be divisible by ngroups"
    assert B.shape == (batch, ngroups, dstate)
    assert C.shape == B.shape
    if D is not None:
        assert D.shape == (nheads, dim)
    if z is not None:
        assert z.shape == x.shape
    if dt_bias is not None:
        assert dt_bias.shape == (nheads, dim)
    
    out = torch.empty_like(x)
    grid = lambda META: (triton.cdiv(dim, META['BLOCK_SIZE_M']), batch, nheads)